In [1]:
import requests
import json
import pandas as pd
import numpy as np
import os
import shutil

In [2]:
def sauvegarde_locale_stations(Npiezmin=10,Nech=20,Npage=1):
    """Sauivegarde en local toutes les infos correspondant à un échantillon de stations"""
    out=requests.get(f"https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/stations?format=json&page={Npage}&size={Nech}")
    stations=json.loads(out.text) # data est la donée json qui est affichée sur le site hubeau,  data['data'] est une liste, de dictionnaires correspondant aux données
    #stations est au format json
    data=pd.json_normalize(stations['data']) # j'extrais data, que je gere en table pandas
    data.loc[data['nb_mesures_piezo']>Npiezmin].to_csv('donnees/stations.csv')

def sauvegarde_locale_chroniques():
    """Met à jour les chroniques à partir de la base de données 'stations' """
    stations=pd.read_csv('donnees/stations.csv',index_col=0)
    try:
        shutil.rmtree('donnees/chroniques') # on vide le dossier chroniques
        print("Dossier donnees/chroniques enlevé")
    except :
        print("On ne peut pas enlever le dossier donnees/chroniques, peut être qu'il n'existe pas")
    try :
        os.mkdir('donnees/chroniques')
        print("Création d'un nouveau dossier donnees/chroniques")
    except :
        print("Pas de création du dossier donnees/chroniques, il existe peut être déja")
    indexes=stations.index.to_numpy()
    #stations.reset_index()
    #print(stations)
    for i in indexes:
        code_bss=stations.loc[i]['code_bss']
        out=requests.get(f"https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss={code_bss}")
        chronique_json=json.loads(out.text)
        chronique=pd.json_normalize(chronique_json['data'])
        chronique.to_csv(f'donnees/chroniques/{i}.csv') #je les renomme avec les index qu'ils ont dans stations, car dans les code bss il y a des "/"
        
def sauvegarde_locale(Npiezmin=10,Nech=20,Npage=1):
    """Appelle les deux fonctions précédentes pour tout faire d'un coup, c'est elle qu'il faut appeller pour refaire completement la sauvegarde locale"""
    try :
        os.mkdir('donnees')
        print("Création d'un dossier donnees")
    except :
        print("Pas de création du dossier donnees, il existe peut être déja")
    sauvegarde_locale_stations(Npiezmin,Nech,Npage)
    sauvegarde_locale_chroniques()

In [3]:
sauvegarde_locale(Nech=100)

Création d'un dossier donnees
On ne peut pas enlever le dossier donnees/chroniques, peut être qu'il n'existe pas
Création d'un nouveau dossier donnees/chroniques


In [6]:
def table_stations():
    """Renvoie la table stations stockée localement si elle existe (servira plus tard de base pour une fonction qui donne la base en regardant si elle existe localement ou pas)"""
    try :
        stations=pd.read_csv('donnees/stations.csv',index_col=0)
    except :
        print("pas de fichier stations.csv")
        return 0
    return stations

In [7]:
table_stations()

,code_bss,urn_bss,date_debut_mesure,date_fin_mesure,code_commune_insee,nom_commune,x,y,codes_bdlisa,urns_bdlisa,...,libelle_pe,profondeur_investigation,codes_masse_eau_edl,noms_masse_eau_edl,urns_masse_eau_edl,date_maj,geometry.type,geometry.crs.type,geometry.crs.properties.name,geometry.coordinates
2,08272X0745/PZ-,http://services.ades.eaufrance.fr/pointeau/082...,1994-01-21,2004-06-21,33039,Bègles,-0.564909,44.812534,NaN,[],...,NaN,6.0,NaN,NaN,[],Tue Apr 21 20:20:08 CEST 2020,Point,name,urn:ogc:def:crs:OGC:1.3:CRS84,"[-0.564909485635284, 44.8125340159062]"
4,12288X0089/PIEZO.,http://services.ades.eaufrance.fr/pointeau/122...,2001-09-20,2020-11-08,97416,Saint-Pierre,55.459616,-21.317968,['974AT01'],['http://reseau.eaufrance.fr/geotraitements/bd...,...,Forage Ravine Blanche (Saint-Pierre - 974),200.0,NaN,NaN,[],Fri Jan 24 16:44:49 CET 2020,Point,name,urn:ogc:def:crs:OGC:1.3:CRS84,"[55.4596161419439, -21.3179680681576]"
5,04454X0089/PK11.0,http://services.ades.eaufrance.fr/pointeau/044...,1966-11-15,1973-12-27,68163,Kembs,7.507685,47.699312,['221AA17'],['http://reseau.eaufrance.fr/geotraitements/bd...,...,NaN,NaN,NaN,NaN,[],Tue Apr 21 20:20:08 CEST 2020,Point,name,urn:ogc:def:crs:OGC:1.3:CRS84,"[7.50768461325435, 47.6993117794598]"
6,04138X0067/PK13.0,http://services.ades.eaufrance.fr/pointeau/041...,1966-11-15,1973-12-27,68238,Niffer,7.521336,47.714658,['221AA17'],['http://reseau.eaufrance.fr/geotraitements/bd...,...,NaN,NaN,NaN,NaN,[],Wed Apr 22 08:04:36 CEST 2020,Point,name,urn:ogc:def:crs:OGC:1.3:CRS84,"[7.52133636700716, 47.7146578509555]"
7,04134X0057/PK23.0,http://services.ades.eaufrance.fr/pointeau/041...,1968-03-22,1973-12-12,68253,Ottmarsheim,7.530655,47.797436,['221AA17'],['http://reseau.eaufrance.fr/geotraitements/bd...,...,NaN,NaN,NaN,NaN,[],Wed Apr 22 08:04:36 CEST 2020,Point,name,urn:ogc:def:crs:OGC:1.3:CRS84,"[7.53065493397775, 47.7974360453209]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,04136X0132/B10,http://services.ades.eaufrance.fr/pointeau/041...,1966-10-07,1984-12-19,68384,Zillisheim,7.300496,47.700743,['226AA03'],['http://reseau.eaufrance.fr/geotraitements/bd...,...,NaN,16.0,NaN,NaN,[],Tue Apr 21 20:20:08 CEST 2020,Point,name,urn:ogc:def:crs:OGC:1.3:CRS84,"[7.30049604478027, 47.7007430502961]"
93,02723X0255/SOC10,http://services.ades.eaufrance.fr/pointeau/027...,1969-05-10,1985-12-16,67482,Strasbourg,7.783675,48.594645,['221AA19'],['http://reseau.eaufrance.fr/geotraitements/bd...,...,NaN,NaN,NaN,NaN,[],Wed Apr 22 20:39:08 CEST 2020,Point,name,urn:ogc:def:crs:OGC:1.3:CRS84,"[7.783675, 48.594645]"
94,08036X2487/PC10,http://services.ades.eaufrance.fr/pointeau/080...,1984-04-04,2004-07-27,33063,Bordeaux,-0.582897,44.851098,NaN,[],...,NaN,7.0,NaN,NaN,[],Wed Apr 22 08:04:36 CEST 2020,Point,name,urn:ogc:def:crs:OGC:1.3:CRS84,"[-0.582897375901089, 44.8510978427418]"
97,03001X0073/D10,http://services.ades.eaufrance.fr/pointeau/030...,1985-04-12,1986-04-16,10372,Soulaines-Dhuys,4.721416,48.381933,NaN,[],...,NaN,12.3,NaN,NaN,[],Wed Apr 22 20:39:08 CEST 2020,Point,name,urn:ogc:def:crs:OGC:1.3:CRS84,"[4.72141620089457, 48.3819326873396]"


In [8]:
def chronique(code_bss):
    """Récupère une chronique à partir du code bss en recoupant à travers les différentes base de données"""
    stations=table_stations()
    if type(stations)=='int' : 
        return 0 # ca a déja affiché l'erreur
    try :
        index=stations[stations['code_bss']==code_bss].index.to_numpy()[0]
        chronique=pd.read_csv(f'donnees/chroniques/{index}.csv',index_col=0)
        print(chronique)
    except :
        print("Il n'y a un probleme pour trouver la chronique")
        
        

In [9]:
chronique('08272X0745/PZ-')

           code_bss                                            urn_bss  \
0    08272X0745/PZ-  http://services.ades.eaufrance.fr/pointeau/082...   
1    08272X0745/PZ-  http://services.ades.eaufrance.fr/pointeau/082...   
2    08272X0745/PZ-  http://services.ades.eaufrance.fr/pointeau/082...   
3    08272X0745/PZ-  http://services.ades.eaufrance.fr/pointeau/082...   
4    08272X0745/PZ-  http://services.ades.eaufrance.fr/pointeau/082...   
..              ...                                                ...   
120  08272X0745/PZ-  http://services.ades.eaufrance.fr/pointeau/082...   
121  08272X0745/PZ-  http://services.ades.eaufrance.fr/pointeau/082...   
122  08272X0745/PZ-  http://services.ades.eaufrance.fr/pointeau/082...   
123  08272X0745/PZ-  http://services.ades.eaufrance.fr/pointeau/082...   
124  08272X0745/PZ-  http://services.ades.eaufrance.fr/pointeau/082...   

    date_mesure  timestamp_mesure  niveau_nappe_eau  mode_obtention  \
0    1994-01-21      759110400000       